✍ В предыдущем юните мы познакомились с теоретической основой случайного леса, а в этом закрепим пройденный материал на практике.

Пожалуй, каждый из нас практически ежедневно смотрит прогноз погоды: будет сегодня тепло или холодно, брать ли зонт? Мы расстраиваемся, если внезапно идёт дождь, а мы оказались к этому не готовы. Иногда можно понять, что будет дождь, просто взглянув на небо, но часто такие предположения оказываются неверными. Надёжнее всего пользоваться прогнозами, которые публикуют специалисты. А задумывались ли вы, как формируются эти прогнозы?

Да-да, здесь тоже не обошлось без машинного обучения. В этом юните мы с вами немного коснёмся метеорологии, чтобы предсказать, будет ли дождь в Австралии. Попутно вы узнаете, какие факторы влияют на вероятность дождя — возможно, вы научитесь предсказывать его точнее, чем метеорологические службы, и больше никогда не окажетесь в нужный момент без зонта.

Давайте посмотрим, с какими данными нам предстоит работать.

In [1]:
import pandas as pd
import numpy as np

from sklearn import tree
from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv("data/weatherAUS.csv", parse_dates=["Date"])
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Date           145460 non-null  datetime64[ns]
 1   Location       145460 non-null  object        
 2   MinTemp        143975 non-null  float64       
 3   MaxTemp        144199 non-null  float64       
 4   Rainfall       142199 non-null  float64       
 5   Evaporation    82670 non-null   float64       
 6   Sunshine       75625 non-null   float64       
 7   WindGustDir    135134 non-null  object        
 8   WindGustSpeed  135197 non-null  float64       
 9   WindDir9am     134894 non-null  object        
 10  WindDir3pm     141232 non-null  object        
 11  WindSpeed9am   143693 non-null  float64       
 12  WindSpeed3pm   142398 non-null  float64       
 13  Humidity9am    142806 non-null  float64       
 14  Humidity3pm    140953 non-null  float64       
 15  

+ Date — дата, в которую зафиксировано наблюдение;
+ Location — местонахождение метеорологической станции;
+ MinTemp — минимальная температура (℃);
+ MaxTemp — максимальная температура (℃);
+ Rainfall — количество осадков (дождь) за сутки (мм);
+ Evaporation — количество испарений до 9 утра (мм);
+ Sunshine — количество часов в сутках, когда светило солнце;
+ WindGustDir — направление самого сильного порыва ветра за последние 24 часа;
+ WindGustSpeed — скорость самого сильного порыва ветра за последние 24 часа;
+ WindDir9am — направление ветра в 9 утра;
+ WindDir3pm — направление ветра в 3 часа дня;
+ WindSpeed9am — скорость ветра в 9 часов утра;
+ WindSpeed3pm — скорость ветра в 3 часа дня;
+ Humidity9am — влажность в 9 утра;
+ Humidity3pm — влажность в 3 часа дня;
+ Pressure9am — атмосферное давление в 9 утра;
+ Pressure3pm — атмосферное давление в 3 часа дня;
+ Cloud9am — часть неба, закрытая облаками, в 9 утра;
+ Cloud3pm — часть неба, закрытая облаками, в 3 часа дня;
+ Temp9am — температура в 9 утра;
+ Temp3pm — температура в 3 часа дня;
+ RainToday — наличие дождя в этот день;
+ RainTomorrow — наличие дождя на следующий день.

Данные содержат 23 признака и 145 460 наблюдений. Из этих 23 признаков шесть — категориальные, в одном записана дата, а остальные являются непрерывными числовыми данными.

+ Примеры числовых признаков: температура, скорость ветра, влажность, облачность, атмосферное давление в разное время суток, количество осадков, испарение, количество часов с солнечной погодой.
+ Примеры категориальных признаков: местоположение, направление ветра в разное время суток, наличие дождя сегодня или завтра.

**Целевой переменной** является столбец *RainTomorrow*. Значение этой переменной мы и будем пытаться предсказать.

## Задание 4.1

Сколько суммарно пропусков в данных?

In [4]:
df.isnull().sum().sum()

343248

Можно отметить, что в данных много пропусков, но для нас это не проблема, так как случайный лес прекрасно работает в таких ситуациях, и скоро у нас будет возможность в этом убедиться.

## Задание 4.2

В некоторых признаках пропусков более 40 % — удалите такие признаки. Сколько их было?

In [5]:
round(df.isna().sum() / len(df), 3)
df.drop(['Evaporation','Sunshine','Cloud3pm'], axis = 1, inplace = True)

## Задание 4.3

Теперь обработаем признаки *RainToday* и *RainTomorrow* таким образом, чтобы вместо *yes* было значение 1, а вместо *no* — значение 0. Обратите внимание на то, что в признаке *RainTomorrow* присутствуют пропуски, и их трогать не нужно, они должны остаться пропусками. Поэтому обрабатывайте столбцы таким образом, чтобы не видоизменить пропущенные значения.

Вычислите среднее арифметическое для преобразованного признака *RainToday* и запишите его в ответ, предварительно округлив до двух знаков после точки-разделителя.

In [6]:
df.RainToday = df.RainToday.map({'No': 0, 'Yes': 1})
df.RainTomorrow = df.RainTomorrow.map({'No': 0, 'Yes': 1})

In [7]:
df['RainToday'].mean()

0.22419285648984874

## Задание 4.4

Обработайте признак Date таким образом, чтобы выделить в отдельный признак Month (номер месяца). Изначальный признак Date удалите. Определите, в какой месяц в среднем за день выпадает больше всего дождей. В качестве ответа введите порядковый номер месяца.

In [8]:
df.Date = pd.to_datetime(df.Date)
df['Month'] = df.Date.dt.month
df.drop('Date', axis = 1, inplace = True)
df_season = df.groupby('Month').mean()
df_season[['RainToday']]

C:\Users\sphex\AppData\Local\Temp\ipykernel_8808\1749877501.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_season = df.groupby('Month').mean()


,RainToday
Month,
1,0.189484
2,0.206746
3,0.217135
4,0.216845
5,0.222163
6,0.263638
7,0.270736
8,0.253167
9,0.229135


## Задание 4.5

Обработайте оставшиеся категориальные признаки. С помощью метода get_dummies с настройками по умолчанию создайте dummy-переменные для всех категориальных признаков (их пять), которые есть в данных на этот момент.

Кодировку признаков важно выполнить именно в следующем порядке: categoricals = ['Month', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']. Это необходимо для того, чтобы ваши дальнейшие ответы сходились с нашим решением, так как алгоритм случайного леса, который мы будем использовать в дальнейшем, чувствителен к порядку столбцов.

Сколько теперь признаков в данных, если считать целевую переменную?

In [9]:
categoricals = ['Month', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']
df_dummies = pd.get_dummies(df, columns=categoricals)
df_dummies.shape

(145460, 124)

## Задание 4.6

Осталось совсем немного. Удалите все строки, где есть пропуски. Далее разбейте данные на обучающую и тестовую выборки в соотношении 70/30, в качестве значения параметра random_state возьмите число 31.

Каково среднее значение целевой переменной на тестовой выборке? Ответ округлите до двух знаков после точки-разделителя.

In [10]:
df_dummies.dropna(inplace=True)
X = df_dummies.drop('RainTomorrow', axis = 1)
y = df_dummies['RainTomorrow']  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 31)
y_test.mean()

0.22770253002811142

## Задание 4.7

Теперь давайте вспомним про бутстреп. Он не понадобится нам для решения этой задачи, но будет полезно реализовать его «вручную».

Сделайте оценку стандартного отклонения для среднего значения минимальной температуры для обучающей выборки (то есть для среднего значения по признаку MinTemp). Для этого сгенерируйте 1000 случайных выборок из наших данных — каждая из них должна быть такого же объёма, как и обучающая выборка. Для генерации выборки используйте np.random.randint(): сгенерируйте необходимое количество индексов и по ним извлеките соответствующие элементы выборки. Случайность фиксируйте с помощью np.random.seed(31).

Для каждой выборки вычислите среднее значение, а после найдите стандартное отклонение для этих значений. Ответ округлите до двух знаков после точки-разделителя.

In [11]:
np.random.seed(31)

samples = X_train["MinTemp"].values[
    np.random.randint(0, len(X_train), (1000, len(X_train)))
]

round(np.std(np.mean(samples, axis=1)), 2)

0.03

## Задание 4.8

Теперь можно перейти к обучению прогностических моделей. Начнём с того, что построим простейшую логистическую регрессию (без настройки гиперпараметров). Это будет та модель, с качеством которой мы будем сравнивать результаты, полученные далее, чтобы оценить превосходство случайного леса над простыми методами.

В качестве ответа введите значение метрики roc_auc на тестовой выборке. Ответ округлите до двух знаков после точки-разделителя.

In [12]:
clf = LogisticRegression()
clf.fit(X_train, y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
roc_auc_score(y_test, preds_test)

c:\Users\sphex\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7274030225534013

## Задание 4.9

In [13]:
params = {
    'max_leaf_nodes': list(range(2, 10)), 
    'min_samples_split': [2, 3, 4], 
    'max_depth': [5,7,9,11]
}

grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, verbose=3, cv=3)
grid_search_cv.fit(X_train, y_train)
print(grid_search_cv.best_params_)

clf = DecisionTreeClassifier(max_depth = 5, max_leaf_nodes = 9, min_samples_split = 2, random_state=42)
clf.fit(X_train, y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
print(round(roc_auc_score(y_test, preds_test), 2))

Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.817 total time=   0.2s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.820 total time=   0.2s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.825 total time=   0.1s
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.817 total time=   0.1s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.820 total time=   0.1s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.825 total time=   0.1s
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.817 total time=   0.1s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.820 total time=   0.1s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.825 total time=   0.1s
[CV 1/3] END max_depth=5, max_leaf_nodes=3, min_samples_split=2;,

## Задание 4.10

In [14]:
clf = RandomForestClassifier(n_estimators = 100, random_state=31)
clf.fit(X_train, y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
print(round(roc_auc_score(y_test, preds_test), 2))

0.73


## Задание 4.11

In [15]:
params = {
    'max_features': [4, 5, 6, 7], 
    'min_samples_leaf': [3, 5, 7, 9, 11], 
    'max_depth': [5, 10, 15]
}
grid_search_cv = GridSearchCV(RandomForestClassifier(random_state=31), params, verbose=3, cv=3)
grid_search_cv.fit(X_train, y_train)
print(grid_search_cv.best_params_)
clf = RandomForestClassifier(max_depth=15, max_features=7, min_samples_leaf=3, random_state=31)
clf.fit(X_train, y_train)
preds_train = clf.predict(X_train)
preds_test = clf.predict(X_test)
print(round(roc_auc_score(y_test, preds_test), 2))

Fitting 3 folds for each of 60 candidates, totalling 180 fits
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.776 total time=   1.6s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.775 total time=   1.6s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.775 total time=   1.4s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.776 total time=   1.5s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.775 total time=   1.5s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.775 total time=   1.9s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.776 total time=   1.7s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.775 total time=   1.7s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.775 total time=   1.4s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=9;, score=0.776 total time=   1.2

## Задание 4.12

In [16]:
feature_names = [x for x in df_dummies if x != 'RainTomorrow']
pd.DataFrame({'feat': feature_names,
              'coef': clf.feature_importances_}).sort_values(by='coef', ascending=False)

,feat,coef
7,Humidity3pm,0.250783
2,Rainfall,0.079757
6,Humidity9am,0.070403
10,Cloud9am,0.067092
9,Pressure3pm,0.065272
...,...,...
50,Location_Newcastle,0.000000
62,Location_SalmonGums,0.000000
51,Location_Nhil,0.000000
52,Location_NorahHead,0.000000


⭐ Итак, вы потренировались решать задачу классификации с помощью случайного леса, рассмотрели на практике некоторые важные свойства этого ансамбля и узнали, от каких факторов зависит, будет ли завтра дождь — весьма полезная информация.

Однако, как вы помните, случайный лес — это не единственный вид ансамблей. Уже в следующем юните мы продолжим наше погружение в этот класс алгоритмов и изучим бустинг →